In [ ]:
# import tensorflow 
# from tensorflow.keras.datasets import cifar10
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import to_categorical

# # Load the CIFAR-10 dataset
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()

# # Normalize the pixel values to be between 0 and 1
# x_train = x_train / 255.0
# x_test = x_test / 255.0

# # Convert the labels to one-hot encoded vectors
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

# # Create a convolutional neural network
# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
#     MaxPooling2D((2, 2)),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D((2, 2)),
#     Conv2D(64, (3, 3), activation='relu'),
#     Flatten(),
#     Dense(64, activation='relu'),
#     Dense(10, activation='softmax')
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Train the model on the training data
# model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# # Evaluate the model on the testing data
# loss, accuracy = model.evaluate(x_test, y_test)

# print(f"Test accuracy: {accuracy}")

In [ ]:
def func(a):								
    for i in range(n):
        a=np.random.randn(n)
        b=np.random.randn(n)
        c=np.dot(a,b)

@jit(target_backend='cuda')
def func2(a):
    for i in range(n):
        A=np.random.randn(n)
        B=np.random.randn(n)
        C=np.dot(A,B)

if __name__=="__main__":
    n=100000
    a=np.random.randn(n)
    # print(a)
    start = timer()
    func(a)
    print("without GPU:", timer()-start)	
	
    start = timer()
    func2(a)
    print("with GPU:", timer()-start)

In [ ]:
from numba import jit, cuda
import numpy as np
# to measure exec time
from timeit import default_timer as timer

# normal function to run on cpu
def func(a):								
	for i in range(1000000):
		a[i]+= 1	

# function optimized to run on gpu
@jit(target_backend='cuda')						
def func2(a):
	for i in range(1000000):
		a[i]+= 1
if __name__=="__main__":
	n = 1000000		
	a = np.ones(n, dtype = np.float64)
	
	start = timer()
	func(a)
	print("without GPU:", timer()-start)	
	
	start = timer()
	func2(a)
	print("with GPU:", timer()-start)


In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jun 14 15:36:26 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.98                 Driver Version: 535.98       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA T1000 8GB             WDDM  | 00000000:01:00.0  On |                  N/A |
| 34%   44C    P0              N/A /  50W |   1003MiB /  8192MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
name = "bigscience/bloom-3b"
text = "Hello my name is"
max_new_tokens = 20

def generate_from_model(model, tokenizer):
  encoded_input = tokenizer(text, return_tensors='pt')
  output_sequences = model.generate(input_ids=encoded_input['input_ids'].cuda())
  return tokenizer.decode(output_sequences[0], skip_special_tokens=True)

In [5]:
from transformers import pipeline

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

name = "bigscience/bloom-3b"
text = "Hello my name is"
max_new_tokens = 20
pipe=''
try:
    tokenizer = AutoTokenizer.from_pretrained(name)
    model = AutoModelForCausalLM.from_pretrained(name)
    pipe = pipeline(model=model, tokenizer=tokenizer, max_new_tokens=max_new_tokens)
    output = pipe(text)
    print(output)
except Exception as e:
    print(f"Error: {e}")


Error: Inferring the task automatically requires to check the hub with a model_id defined as a `str`.BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 2560)
    (word_embeddings_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=T